In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/googlenewsvectors/GoogleNews-vectors-negative300.bin
/kaggle/input/2-recommended-reads-conversion-of-data-to-num/__results__.html
/kaggle/input/2-recommended-reads-conversion-of-data-to-num/vectorizedData.csv
/kaggle/input/2-recommended-reads-conversion-of-data-to-num/__notebook__.ipynb
/kaggle/input/2-recommended-reads-conversion-of-data-to-num/__output__.json
/kaggle/input/2-recommended-reads-conversion-of-data-to-num/custom.css
/kaggle/input/2-recommended-reads-conversion-of-data-to-num/__results___files/__results___9_0.png


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from gensim.models.keyedvectors import KeyedVectors
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


# Load the keyed vectors
word_vectors = KeyedVectors.load_word2vec_format('/kaggle/input/googlenewsvectors/GoogleNews-vectors-negative300.bin', binary=True)

# Load the data
data = pd.read_csv("/kaggle/input/2-recommended-reads-conversion-of-data-to-num/vectorizedData.csv")
data = data.drop_duplicates(subset=['booktitle', 'authorname'], keep='first')

# Tokenize and vectorize the first book description
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['bookdescription'])
sequences = tokenizer.texts_to_sequences(data['bookdescription'])
word_index = tokenizer.word_index
max_len = 100 # set the maximum length of the sequence
X_desc = pad_sequences(sequences, maxlen=max_len)
X_vec = np.zeros((len(data), word_vectors.vector_size))
for i, desc in enumerate(data['bookdescription']):
    words = desc.split()
    vectors = [word_vectors[word] for word in words if word in word_vectors]
    if vectors:
        X_vec[i] = np.mean(vectors, axis=0)

# Combine the tokenized and vectorized features
X = np.concatenate((X_desc, X_vec), axis=1)

# Encoding the type column
label_encoder = LabelEncoder()
label_encoder.fit(data['type'])
y = label_encoder.transform(data['type'])

# Splitting the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Creating and training the model
model = Sequential()
model.add(Dense(512, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(216, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, pd.get_dummies(y_train), epochs=50, batch_size=32, validation_data=(X_test, pd.get_dummies(y_test)))


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Epoch 1/50
276/276 [==============================] - 4s 8ms/step - loss: 53.6088 - accuracy: 0.1079 - val_loss: 2.4653 - val_accuracy: 0.1066
Epoch 2/50
276/276 [==============================] - 2s 6ms/step - loss: 2.4605 - accuracy: 0.1094 - val_loss: 2.4547 - val_accuracy: 0.1066
Epoch 3/50
276/276 [==============================] - 2s 6ms/step - loss: 2.4531 - accuracy: 0.1134 - val_loss: 2.4493 - val_accuracy: 0.1066
Epoch 4/50
276/276 [==============================] - 2s 6ms/step - loss: 2.4496 - accuracy: 0.1104 - val_loss: 2.4467 - val_accuracy: 0.1066
Epoch 5/50
276/276 [==============================] - 2s 6ms/step - loss: 2.4479 - accuracy: 0.1134 - val_loss: 2.4453 - val_accuracy: 0.1066
Epoch 6/50
276/276 [==============================] - 2s 6ms/step - loss: 2.4471 - accuracy: 0.1134 - val_loss: 2.4446 - val_accuracy: 0.1066
Epoch 7/50
276/276 [==============================] - 2s 7ms/step - loss: 2.4467 - accuracy: 0.1134 - val_loss: 2.4442 - val_accuracy: 0.1066
Epoch

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Conv1D, GlobalMaxPooling1D
from sklearn.preprocessing import LabelEncoder
from gensim.models.keyedvectors import KeyedVectors
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


# Load the keyed vectors
word_vectors = KeyedVectors.load_word2vec_format('/kaggle/input/googlenewsvectors/GoogleNews-vectors-negative300.bin', binary=True)

# Load the data
data = pd.read_csv("/kaggle/input/2-recommended-reads-conversion-of-data-to-num/vectorizedData.csv")
data = data.drop_duplicates(subset=['booktitle', 'authorname'], keep='first')

# Tokenize and vectorize the first book description
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['bookdescription'])
sequences = tokenizer.texts_to_sequences(data['bookdescription'])
word_index = tokenizer.word_index
max_len = 100 # set the maximum length of the sequence
X_desc = pad_sequences(sequences, maxlen=max_len)
X_vec = np.zeros((len(data), word_vectors.vector_size))
for i, desc in enumerate(data['bookdescription']):
    words = desc.split()
    vectors = [word_vectors[word] for word in words if word in word_vectors]
    if vectors:
        X_vec[i] = np.mean(vectors, axis=0)

# Combine the tokenized and vectorized features
X = np.concatenate((X_desc, X_vec), axis=1)

# Encoding the type column
label_encoder = LabelEncoder()
label_encoder.fit(data['type'])
y = label_encoder.transform(data['type'])

# Splitting the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Creating and training the model
model = Sequential()
model.add(Conv1D(128, kernel_size=5, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(Conv1D(64, kernel_size=3, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(len(label_encoder.classes_), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train.reshape((len(X_train), X_train.shape[1], 1)), pd.get_dummies(y_train), epochs=50, batch_size=32, validation_data=(X_test.reshape((len(X_test), X_test.shape[1], 1)), pd.get_dummies(y_test)))


Epoch 1/50
276/276 [==============================] - 14s 48ms/step - loss: 538.8013 - accuracy: 0.0893 - val_loss: 294.2744 - val_accuracy: 0.0744
Epoch 2/50
276/276 [==============================] - 13s 47ms/step - loss: 279.6488 - accuracy: 0.0930 - val_loss: 195.4376 - val_accuracy: 0.0957
Epoch 3/50
276/276 [==============================] - 12s 43ms/step - loss: 194.0098 - accuracy: 0.0938 - val_loss: 162.2690 - val_accuracy: 0.0943
Epoch 4/50
276/276 [==============================] - 12s 43ms/step - loss: 167.2136 - accuracy: 0.0959 - val_loss: 168.7647 - val_accuracy: 0.0857
Epoch 5/50
276/276 [==============================] - 11s 41ms/step - loss: 112.7976 - accuracy: 0.0912 - val_loss: 68.5728 - val_accuracy: 0.1020
Epoch 6/50
276/276 [==============================] - 11s 41ms/step - loss: 64.4155 - accuracy: 0.0890 - val_loss: 24.3817 - val_accuracy: 0.1020
Epoch 7/50
276/276 [==============================] - 12s 42ms/step - loss: 5.7394 - accuracy: 0.1077 - val_loss: 2